<a href="https://colab.research.google.com/github/dema-u/keras_triplet_descriptor/blob/master/BaselineCodeTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### GPU CHECKS AND IMPORTING DATA

In [1]:
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
!pip install talos
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# Colab only provides one GPU and it is not always guaranteed
gpu = GPUs[0]
def printm():
  process = psutil.Process(os.getpid())
  print("RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
  print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
  
printm()

import sys
import json
import os
import glob
import time
import tensorflow as tf
import numpy as np
import cv2
import random
import talos
import pandas as pd
import matplotlib.pyplot as plt

import keras
from keras import backend as K
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten, Input, Lambda, Reshape
from keras.layers import Conv2D, Conv3D, MaxPooling2D, BatchNormalization 
from keras.layers import Input, UpSampling2D, concatenate  
from keras.activations import sigmoid, relu, elu
from keras.layers.convolutional import Deconvolution2D
from talos import live, Reporting
from talos.model.normalizers import lr_normalizer
from keras.losses import mean_squared_error, binary_crossentropy

!git clone https://github.com/MatchLab-Imperial/keras_triplet_descriptor
  
%cd /content/keras_triplet_descriptor  

!wget -O hpatches_data.zip https://imperialcollegelondon.box.com/shared/static/ah40eq7cxpwq4a6l4f62efzdyt8rm3ha.zip
  
!unzip -q ./hpatches_data.zip
!rm ./hpatches_data.zip

hpatches_dir = './hpatches'
splits_path = './splits.json'

splits_json = json.load(open(splits_path, 'rb'))
split = splits_json['a']

train_fnames = split['train']
test_fnames = split['test']

seqs = glob.glob(hpatches_dir+'/*')
seqs = [os.path.abspath(p) for p in seqs]   
seqs_train = list(filter(lambda x: x.split('/')[-1] in train_fnames, seqs)) 
seqs_test = list(filter(lambda x: x.split('/')[-1] in split['test'], seqs))

from read_data import HPatches, DataGeneratorDesc, hpatches_sequence_folder, DenoiseHPatches, tps
from utils import generate_desc_csv, plot_denoise, plot_triplet

random.seed(1234)
np.random.seed(1234)
tf.set_random_seed(1234)

('RAM Free: 11.5 GB', ' | Proc size: 151.4 MB')
GPU RAM Free: 9257MB | Used: 2184MB | Util  19% | Total 11441MB


/usr/local/lib/python2.7/dist-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools
/usr/local/lib/python2.7/dist-packages/chances/plots.py:7: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/usr/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python2.7/d

fatal: destination path 'keras_triplet_descriptor' already exists and is not an empty directory.
/content/keras_triplet_descriptor
--2019-03-07 17:13:11--  https://imperialcollegelondon.box.com/shared/static/ah40eq7cxpwq4a6l4f62efzdyt8rm3ha.zip
Resolving imperialcollegelondon.box.com (imperialcollegelondon.box.com)... 185.235.236.197
Connecting to imperialcollegelondon.box.com (imperialcollegelondon.box.com)|185.235.236.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/ah40eq7cxpwq4a6l4f62efzdyt8rm3ha.zip [following]
--2019-03-07 17:13:11--  https://imperialcollegelondon.box.com/public/static/ah40eq7cxpwq4a6l4f62efzdyt8rm3ha.zip
Reusing existing connection to imperialcollegelondon.box.com:443.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://imperialcollegelondon.app.box.com/public/static/ah40eq7cxpwq4a6l4f62efzdyt8rm3ha.zip [following]
--2019-03-07 17:13:12--  https://imperialcollegelondon.app.b

### TESTING BASELINE CODE

In [2]:
denoise_generator = DenoiseHPatches(random.sample(seqs_train, 3), batch_size=50)
denoise_generator_val = DenoiseHPatches(random.sample(seqs_test, 1), batch_size=50)

100%|██████████| 1/1 [00:02<00:00,  2.59s/it]


In [0]:
def get_denoise_model_baseline(shape):
    
  inputs = Input(shape)
  
  ## Encoder starts
  conv1 = Conv2D(16, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
  pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
  
  ## Bottleneck
  conv2 = Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)

  up3 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv2))
  merge3 = concatenate([conv1,up3], axis = -1)
  conv3 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge3)
    
  conv4 = Conv2D(1, 3,  padding = 'same')(conv3)

  shallow_net = Model(inputs = inputs, outputs = conv4)
  
  return shallow_net

In [0]:
def get_denoise_model_baseline(shape):
    
  inputs = Input(shape)
  
  conv1 = Conv2D(16, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
  pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
  conv2 = Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
  pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
  conv3 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
  
  up3 = Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv3))
  merge3 = concatenate([conv2,up3], axis = -1)
  up4 = Conv2D(16, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(merge3))
  merge4 = concatenate([conv1,up4], axis = -1)
  conv4 = Conv2D(16, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge4)
    
  conv5 = Conv2D(1, 3,  padding = 'same')(conv4)

  shallow_net = Model(inputs = inputs, outputs = conv5)
  
  return shallow_net

In [18]:
get_denoise_model_baseline((32, 32, 1)).summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            (None, 32, 32, 1)    0                                            
__________________________________________________________________________________________________
conv2d_37 (Conv2D)              (None, 32, 32, 16)   160         input_7[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_10 (MaxPooling2D) (None, 16, 16, 16)   0           conv2d_37[0][0]                  
__________________________________________________________________________________________________
conv2d_38 (Conv2D)              (None, 16, 16, 32)   4640        max_pooling2d_10[0][0]           
__________________________________________________________________________________________________
up_samplin

In [19]:
try: del denoise_model_baseline; print("Deleted previous model. Compiling a new one...")
except: print("The model has not been initialized. Compiling...")

shape = (32, 32, 1)
denoise_model_baseline = get_denoise_model_baseline(shape)

sgd = keras.optimizers.SGD(lr=0.00001, momentum=0.9, nesterov=True)
denoise_model_baseline.compile(loss='mean_absolute_error', optimizer=sgd, metrics=['mae'])

Deleted previous model. Compiling a new one...


In [0]:
denoise_history_baseline = denoise_model_baseline.fit_generator(generator=denoise_generator, epochs=10, verbose=1, validation_data=denoise_generator_val)

Epoch 1/10
1718/1718 [==============================] - 61s 35ms/step - loss: 9.9069 - mean_absolute_error: 9.9069 - val_loss: 7.3893 - val_mean_absolute_error: 7.3893
Epoch 2/10
 816/1718 [=============>................] - ETA: 30s - loss: 8.1179 - mean_absolute_error: 8.1179

In [0]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(2,1)
axes[0].plot(denoise_history_baseline.history['mean_absolute_error'], 'r')
axes[0].set_title('De-noise Model Training Loss')
axes[0].set_ylabel('Mean Average Error (%)')
axes[0].set_xlabel('Number of epochs')

fig.subplots_adjust(hspace=0.5)

axes[1].plot(denoise_history_baseline.history['val_mean_absolute_error'], 'g')
axes[1].set_title('De-noise Model Validation Loss')
axes[1].set_ylabel('Mean Average Error (%)')
axes[1].set_xlabel('Number of epochs')

In [0]:
hPatches = HPatches(train_fnames=train_fnames, test_fnames=test_fnames, denoise_model=denoise_model_baseline, use_clean=False)
training_generator = DataGeneratorDesc(*hPatches.read_image_file(hpatches_dir, train=1), num_triplets=100000)
val_generator = DataGeneratorDesc(*hPatches.read_image_file(hpatches_dir, train=0), num_triplets=10000)

In [0]:
def get_descriptor_model_baseline(shape):
  
  '''Architecture copies HardNet architecture'''
  
  init_weights = keras.initializers.he_normal()
  
  descriptor_model = Sequential()
  descriptor_model.add(Conv2D(32, 3, padding='same', input_shape=shape, use_bias = True, kernel_initializer=init_weights))
  descriptor_model.add(BatchNormalization(axis = -1))
  descriptor_model.add(Activation('relu'))

  descriptor_model.add(Conv2D(32, 3, padding='same', use_bias = True, kernel_initializer=init_weights))
  descriptor_model.add(BatchNormalization(axis = -1))
  descriptor_model.add(Activation('relu'))

  descriptor_model.add(Conv2D(64, 3, padding='same', strides=2, use_bias = True, kernel_initializer=init_weights))
  descriptor_model.add(BatchNormalization(axis = -1))
  descriptor_model.add(Activation('relu'))

  descriptor_model.add(Conv2D(64, 3, padding='same', use_bias = True, kernel_initializer=init_weights))
  descriptor_model.add(BatchNormalization(axis = -1))
  descriptor_model.add(Activation('relu'))

  descriptor_model.add(Conv2D(128, 3, padding='same', strides=2,  use_bias = True, kernel_initializer=init_weights))
  descriptor_model.add(BatchNormalization(axis = -1))
  descriptor_model.add(Activation('relu'))

  descriptor_model.add(Conv2D(128, 3, padding='same', use_bias = True, kernel_initializer=init_weights))
  descriptor_model.add(BatchNormalization(axis = -1))
  descriptor_model.add(Activation('relu'))
  descriptor_model.add(Dropout(0.3))

  descriptor_model.add(Conv2D(128, 8, padding='valid', use_bias = True, kernel_initializer=init_weights))
  
  descriptor_model.add(Reshape((128,)))
  
  return descriptor_model

def triplet_loss_baseline(x):
  
  output_dim = 128
  a, p, n = x
  _alpha = 1.0
  positive_distance = K.mean(K.square(a - p), axis=-1)
  negative_distance = K.mean(K.square(a - n), axis=-1)
  
  return K.expand_dims(K.maximum(0.0, positive_distance - negative_distance + _alpha), axis = 1)

In [0]:
try: del descriptor_model_baseline; print("Deleted previous model. Compiling a new one...")
except: print("The model has not been initialized. Compiling...")

shape = (32, 32, 1)
xa = Input(shape=shape, name='a')
xp = Input(shape=shape, name='p')
xn = Input(shape=shape, name='n')
descriptor_model_baseline = get_descriptor_model_baseline(shape)
ea = descriptor_model_baseline(xa)
ep = descriptor_model_baseline(xp)
en = descriptor_model_baseline(xn)

loss = Lambda(triplet_loss_baseline)([ea, ep, en])

descriptor_model_baseline = Model(inputs=[xa, xp, xn], outputs=loss)

sgd = keras.optimizers.SGD(lr=0.1)
descriptor_model_baseline.compile(loss='mean_absolute_error', optimizer=sgd)

In [0]:
descriptor_history_baseline = descriptor_model_baseline.fit_generator(generator=training_generator, epochs=10, verbose=1, validation_data=val_generator)

In [0]:
fig, axes = plt.subplots(2,1)
axes[0].plot(descriptor_history_baseline.history['loss'], 'r')
axes[0].set_title('De-noise Model Training Loss')
axes[0].set_ylabel('Error (%)')
axes[0].set_xlabel('Number of epochs')

fig.subplots_adjust(hspace=0.5)

axes[1].plot(descriptor_history_baseline.history['val_loss'], 'g')
axes[1].set_title('De-noise Model Validation Loss')
axes[1].set_ylabel('Error (%)')
axes[1].set_xlabel('Number of epochs')

In [0]:
generate_desc_csv(descriptor_model_baseline, seqs_test, denoise_model = denoise_model_baseline, use_clean=False)

!python ./hpatches-benchmark/hpatches_eval.py --descr-name=custom --descr-dir=/content/keras_triplet_descriptor/out/ --task=verification --delimiter=";"
!python ./hpatches-benchmark/hpatches_results.py --descr=custom --results-dir=./hpatches-benchmark/results/ --task=verification

!python ./hpatches-benchmark/hpatches_eval.py --descr-name=custom --descr-dir=/content/keras_triplet_descriptor/out/ --task=matching --delimiter=";"
!python ./hpatches-benchmark/hpatches_results.py --descr=custom --results-dir=./hpatches-benchmark/results/ --task=matching

!python ./hpatches-benchmark/hpatches_eval.py --descr-name=custom --descr-dir=/content/keras_triplet_descriptor/out/ --task=retrieval --delimiter=";"
!python ./hpatches-benchmark/hpatches_results.py --descr=custom --results-dir=./hpatches-benchmark/results/ --task=retrieval